In [ ]:
"""qiskit_runtime.ipynb"""

from pathlib import Path

from qiskit_ibm_runtime import QiskitRuntimeService

# Cell 1 - Save IBM API Token

my_api_token = "_your_api_key_goes_here_"

# Saves your IBM Quantum credential to $HOME/.qiskit/qiskit-ibm.json
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token=my_api_token,
    overwrite=True,
)

# Verify credential file was created
credential_file = open(Path.home() / ".qiskit" / "qiskit-ibm.json", "r").read()
print(credential_file)

In [ ]:
# Cell 2 - Display available primitives
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")
service.pprint_programs()

In [ ]:
# Cell 3 - Run a test reference job
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")
program_inputs = {"iterations": 1}
options = {"backend": "ibmq_qasm_simulator"}
job = service.run(program_id="hello-world", options=options, inputs=program_inputs)

print(job.status())
print(f"job id: {job.job_id()}")

result = job.result()
print(result)

In [ ]:
# Cell 4 - Display available backends
from qiskit_ibm_runtime import QiskitRuntimeService

for backend in service.backends(simulator=False, operational=True):
    config = backend.configuration()
    print(
        f"{config.backend_name:15}: Qubits = {config.n_qubits}: Gates = {config.basis_gates}"
    )

In [ ]:
# Cell 5 - Run a test circuit on a real backend

from qiskit import QuantumCircuit
from qiskit_ibm_runtime import Options, QiskitRuntimeService, Sampler, Session

# Create circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

display(qc.draw("mpl"))

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_belem")

options = Options()
options.optimization_level = 3  # adds dynamical decoupling
options.resilience_level = 1  # adds readout error mitigation
options.execution.shots = 1024

sampler = Sampler(session=backend, options=options)
job = sampler.run(qc)
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

result = job.result()
print(f">>> {result}")
print(f"  > Quasi-distribution: {result.quasi_dists[0]}")
print(f"  > Metadata: {result.metadata[0]}")

In [ ]:
# Cell 6 - Plot results

from qiskit.tools.visualization import plot_histogram

plot_histogram(result.quasi_dists)